In [0]:
from numpy import array
import keras
from keras.callbacks import CSVLogger 
history = History()
import numpy as np
import matplotlib.pyplot as plt
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
import io

In [0]:
# load doc into memory
def load_doc(filename):
    with io.open(filename, 'r', encoding='utf8') as f:
        text = f.read()
    return text

# load
in_filename = r'updated_data_5_1.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [71]:
lines

['\ufeffকর্পোরেশনের দোষ তো আছে । কিন্তু',
 'দোষ তো আছে । কিন্তু আমরা',
 'তো আছে । কিন্তু আমরা যে',
 'আছে । কিন্তু আমরা যে সুযোগ',
 '। কিন্তু আমরা যে সুযোগ বুঝে',
 'কিন্তু আমরা যে সুযোগ বুঝে হাজার',
 'আমরা যে সুযোগ বুঝে হাজার হাজার',
 'যে সুযোগ বুঝে হাজার হাজার কোটি',
 'সুযোগ বুঝে হাজার হাজার কোটি টাকার',
 'বুঝে হাজার হাজার কোটি টাকার ট্যাক্স',
 'হাজার হাজার কোটি টাকার ট্যাক্স ফাকি',
 'হাজার কোটি টাকার ট্যাক্স ফাকি দিয়ে',
 'কোটি টাকার ট্যাক্স ফাকি দিয়ে আসছি',
 'টাকার ট্যাক্স ফাকি দিয়ে আসছি সেটাও',
 'ট্যাক্স ফাকি দিয়ে আসছি সেটাও আমরা',
 'ফাকি দিয়ে আসছি সেটাও আমরা জানি',
 'দিয়ে আসছি সেটাও আমরা জানি ।',
 'আসছি সেটাও আমরা জানি । এই',
 'সেটাও আমরা জানি । এই ভয়ে',
 'আমরা জানি । এই ভয়ে সবাই',
 'জানি । এই ভয়ে সবাই স্যু',
 '। এই ভয়ে সবাই স্যু করার',
 'এই ভয়ে সবাই স্যু করার সাহস',
 'ভয়ে সবাই স্যু করার সাহস পাবেনা',
 'সবাই স্যু করার সাহস পাবেনা ।',
 'স্যু করার সাহস পাবেনা । কারণ',
 'করার সাহস পাবেনা । কারণ উলটা',
 'সাহস পাবেনা । কারণ উলটা স্যু',
 'পাবেনা । কারণ উলটা স্যু সে',
 '। কারণ উলটা স্যু সে

In [0]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [73]:
sequences

[[10369, 2568, 49, 20, 1, 18],
 [2568, 49, 20, 1, 18, 92],
 [49, 20, 1, 18, 92, 17],
 [20, 1, 18, 92, 17, 280],
 [1, 18, 92, 17, 280, 512],
 [18, 92, 17, 280, 512, 400],
 [92, 17, 280, 512, 400, 400],
 [17, 280, 512, 400, 400, 3419],
 [280, 512, 400, 400, 3419, 1379],
 [512, 400, 400, 3419, 1379, 3420],
 [400, 400, 3419, 1379, 3420, 3421],
 [400, 3419, 1379, 3420, 3421, 45],
 [3419, 1379, 3420, 3421, 45, 1380],
 [1379, 3420, 3421, 45, 1380, 669],
 [3420, 3421, 45, 1380, 669, 92],
 [3421, 45, 1380, 669, 92, 456],
 [45, 1380, 669, 92, 456, 1],
 [1380, 669, 92, 456, 1, 4],
 [669, 92, 456, 1, 4, 1950],
 [92, 456, 1, 4, 1950, 135],
 [456, 1, 4, 1950, 135, 1381],
 [1, 4, 1950, 135, 1381, 84],
 [4, 1950, 135, 1381, 84, 670],
 [1950, 135, 1381, 84, 670, 3422],
 [135, 1381, 84, 670, 3422, 1],
 [1381, 84, 670, 3422, 1, 153],
 [84, 670, 3422, 1, 153, 3423],
 [670, 3422, 1, 153, 3423, 1381],
 [3422, 1, 153, 3423, 1381, 11],
 [1, 153, 3423, 1381, 11, 610],
 [153, 3423, 1381, 11, 610, 382],
 [3423, 

In [0]:
length = 6

In [75]:
print(len(sequences))
print(sequences[0][0])

60233
10369


In [76]:
sequences = array(sequences)
sequences

array([list([10369, 2568, 49, 20, 1, 18]),
       list([2568, 49, 20, 1, 18, 92]), list([49, 20, 1, 18, 92, 17]),
       ..., list([1, 2375, 1329, 484, 1, 10368]),
       list([2375, 1329, 484, 1, 10368, 4]),
       list([1329, 484, 1, 10368, 4, 10370])], dtype=object)

In [77]:
c=1
h=0
while(c!=0):
    limit = sequences.size
    d =[]
    c=0
    for i in range(0 , limit):
        l=len(sequences[i])
        if(l!=length):
            print(sequences[i])
            sequences = np.delete(sequences, i, 0)
            
            c=1
            print("error at ", i)
            print("Deleting :_________________________ ") 
            h = h+1
            break        
print("Total deletion: ", h)     

[330, 70, 7, 979, 20]
error at  6665
Deleting :_________________________ 
[70, 7, 979, 20, 197]
error at  6665
Deleting :_________________________ 
[7, 979, 20, 197, 6]
error at  6665
Deleting :_________________________ 
[979, 20, 197, 6, 237]
error at  6665
Deleting :_________________________ 
[20, 197, 6, 237, 62]
error at  6665
Deleting :_________________________ 
[197, 6, 237, 62, 255]
error at  6665
Deleting :_________________________ 
[618, 593, 10, 85, 148]
error at  7715
Deleting :_________________________ 
[593, 10, 85, 148, 6]
error at  7715
Deleting :_________________________ 
[10, 85, 148, 6, 4032]
error at  7715
Deleting :_________________________ 
[85, 148, 6, 4032, 4033]
error at  7715
Deleting :_________________________ 
[148, 6, 4032, 4033, 701]
error at  7715
Deleting :_________________________ 
[6, 4032, 4033, 701, 1]
error at  7715
Deleting :_________________________ 
[174, 1267, 69, 2, 24]
error at  7817
Deleting :_________________________ 
[1267, 69, 2, 24, 24]
er

In [78]:
print(sequences[564])
print(sequences.size)

[2591, 238, 1394, 1, 37, 514]
57842


In [79]:
sequences = np.array(sequences.tolist())
sequences 

array([[10369,  2568,    49,    20,     1,    18],
       [ 2568,    49,    20,     1,    18,    92],
       [   49,    20,     1,    18,    92,    17],
       ...,
       [    1,  2375,  1329,   484,     1, 10368],
       [ 2375,  1329,   484,     1, 10368,     4],
       [ 1329,   484,     1, 10368,     4, 10370]])

In [80]:
sequences.shape

(57842, 6)

In [0]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [0]:
class My_Callback(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        def on_epoch_end(self, epoch, logs=None):
          logs = logs or {}
        for k in self.params['metrics']:
            if k in logs:
                print("Name: %s, Value: %s" % (k, logs[k]))
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        return

In [83]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=seq_length))
model.add(GRU(100, return_sequences=True))
model.add(GRU(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
csv_logger = CSVLogger('training_5_1.csv')
model.fit(X, y, batch_size=100, epochs=500,validation_split= 0.2 , callbacks=[csv_logger])

57842/57842 [==============================] - 5s 82us/step - loss: 0.0135 - acc: 0.9971
Epoch 479/1000
57842/57842 [==============================] - 5s 81us/step - loss: 0.0136 - acc: 0.9970
Epoch 480/1000
57842/57842 [==============================] - 5s 81us/step - loss: 0.0134 - acc: 0.9972
Epoch 481/1000
57842/57842 [==============================] - 5s 81us/step - loss: 0.0135 - acc: 0.9972
Epoch 482/1000
57842/57842 [==============================] - 5s 81us/step - loss: 0.0137 - acc: 0.9970
Epoch 483/1000
57842/57842 [==============================] - 5s 81us/step - loss: 0.0135 - acc: 0.9971
Epoch 484/1000
57842/57842 [==============================] - 5s 81us/step - loss: 0.0139 - acc: 0.9971
Epoch 485/1000
57842/57842 [==============================] - 5s 81us/step - loss: 0.0142 - acc: 0.9970
Epoch 486/1000
57842/57842 [==============================] - 5s 81us/step - loss: 0.0140 - acc: 0.9971
Epoch 487/1000
57842/57842 [==============================] - 5s 81us/step - lo

In [0]:
# save the model to file
model.save('model_5_1.h5')
#model.save(r'C:\Users\OFR\Desktop\Thesis work\version three\model_5_1.h5')
# save the tokenizer
dump(tokenizer, open(r'tokenizer_5_1.pkl', 'wb'))